# Markov Decision Process

## Section 1: Introduction

In this section, we will introduce the concepts of MDP, Q-values, and V-values. These concepts are fundamental to the field of AI and machine learning, as they are used to model **decision-making problems** in various domains such as "robotics", "finance", and "healthcare".

MDP stands for Markov Decision Process. It is a mathematical framework for modeling decision-making problems in which the outcomes are partly random and partly under the control of a decision-maker. MDPs are defined by a set of states, a set of actions, a reward function, and a transition function. The goal is to find a policy that maximizes the expected cumulative reward over time.

Q-values and V-values are two important concepts in the context of MDPs. A Q-value represents the expected cumulative reward of taking a particular action in a particular state and following a specific policy thereafter. A V-value represents the expected cumulative reward of being in a particular state and following a specific policy thereafter. These values are used to evaluate and improve the policy of an agent in an MDP.

## Section 2: The Basics of MDPs

In this section, we will explain the basic components of an MDP.

An MDP is defined by "a set of states", "a set of actions", "a reward function", and "a transition function". The state space is the set of all possible states that the agent can be in. The action space is the set of all possible actions that the agent can take. The reward function defines the reward the agent receives for each action taken in a particular state. The transition function defines the probability of moving from one state to another state after taking a particular action.

To illustrate these concepts, let's consider an example of a **robot that needs to navigate through a maze**. The robot can be in one of several states, such as at the start of the maze, at a junction in the maze, or at the end of the maze. This robot takes an action. With Probability of **0.8** It goes in that desired direction but with probability of **0.2** It goes in the perpendicular direction (0.1, 0.1 for each)!

In an MDP, the agent interacts with the environment by selecting actions based on its current state and the expected future reward. The goal of the agent is to find a policy that maximizes the expected cumulative reward over time.

**QUESTION**

1. What are the state space, action space, reward function, and transition function of the robot in the maze example? Explain why you think each of these components is important for the robot to navigate through the maze.

Answer:
The state space in the maze example would consist of all possible locations or configurations of the robot within the maze. It includes states such as being at the start of the maze, at various junctions, or at the end of the maze. The state space is important because it provides the robot with the necessary information about its current position, allowing it to make decisions based on its location.

The action space represents the available actions that the robot can take at any given state. In the maze example, these actions could include moving in different directions such as up, down, left, or right. The action space is crucial as it provides the robot with the options to navigate through the maze and progress towards the goal.

The reward function assigns a numerical reward to the robot based on its actions and states. In the maze example, the reward function would likely provide positive rewards for reaching the goal and negative rewards for colliding with obstacles or deviating from the desired path. The reward function is important as it serves as feedback for the robot's decision-making process, guiding it towards actions that maximize cumulative rewards.

The transition function describes the dynamics of the environment and determines the probability of transitioning from one state to another after taking a specific action. In the maze example, the transition function would specify the probability of moving in the desired direction (0.8) or in a perpendicular direction (0.2) after selecting an action. The transition function is essential because it captures the uncertainty in the environment and allows the robot to account for the possible outcomes of its actions.

2. Is our environment stochastic or deterministic? Why?!

Answer:
Based on the information provided, the environment in this maze example is stochastic. This is because there is a probabilistic element involved in the transition from one state to another after taking an action. Specifically, there is a 0.8 probability of moving in the desired direction and a 0.2 probability of moving in a perpendicular direction. The stochastic nature of the environment introduces uncertainty and adds complexity to the robot's decision-making process, requiring it to consider the potential outcomes of its actions and make informed choices.

**Define The MDP**:

Based on your choice of rewards and transitions and the state space, define the MDP for the robot in the maze example. You can complete the following code to define the MDP:

In [3]:
import numpy as np
maze = np.array([[2, 0, 0, 0, 0],
                 [0, 1, 1, 0, 1],
                 [0, 0, 0, 0, 0],
                 [0, 1, 1, 1, 3]])
print("Our Maze is: \n", maze)
states = [i for i in range(maze.shape[0]*maze.shape[1])]
actions = {0:'up', 1:'down', 2:'left', 3:'right'}
rewards = [0 if i!=3 or j!=4 else 10 for i in range(4) for j in range (5) ]
transition_probs = [[[0 for i in range (len(actions))] for j in range(len(states))] for k in range (len(states))]
for i in range(len(states)):
    if (i+1)%5 != 0 and maze[i//5][(i%5)+1]!=1:
        transition_probs[i][i+1][3]=0.8
        transition_probs[i][i+1][0]=0.1
        transition_probs[i][i+1][1]=0.1
    else:
        transition_probs[i][i][3]=0.8
        transition_probs[i][i][0]=0.1
        transition_probs[i][i][1]=0.1
    if i%5 != 0 and maze[i//5][(i%5)-1]!=1:
        transition_probs[i][i-1][2]=0.8
        transition_probs[i][i-1][0]=0.1
        transition_probs[i][i-1][1]=0.1
    else:
        transition_probs[i][i][2]=0.8
        transition_probs[i][i][0]=0.1
        transition_probs[i][i][1]=0.1
    if i>4 and maze[(i//5)-1][(i%5)]!=1:
        transition_probs[i][i-5][2]=0.1
        transition_probs[i][i-5][0]=0.8
        transition_probs[i][i-5][3]=0.1
    else:
        transition_probs[i][i][2]=0.1
        transition_probs[i][i][0]=0.8
        transition_probs[i][i][3]=0.1
    if i<15 and maze[(i//5)+1][(i%5)]!=1:
        transition_probs[i][i+5][2]=0.1
        transition_probs[i][i+5][1]=0.8
        transition_probs[i][i+5][3]=0.1
    else:
        transition_probs[i][i][2]=0.1
        transition_probs[i][i][1]=0.8
        transition_probs[i][i][3]=0.1
discount = 0.9
values = {state: 0 for state in states}
q_values = {(state, action): 0 for state in states for action in actions}

Our Maze is: 
 [[2 0 0 0 0]
 [0 1 1 0 1]
 [0 0 0 0 0]
 [0 1 1 1 3]]


## Section 3: Computing V-values

In this section, we will explain how to compute V-values for an MDP using the Bellman equation.

The Bellman equation is a recursive equation that expresses the value of a state in terms of the values of its successor states. It is defined as:

$$V(s) = R(s) + \gamma * \max_a (\sum_{s'} P(s, a, s') * V(s'))$$

where V(s) is the value of state s, R(s) is the reward for being in state s, γ is the discount factor that determines the importance of future rewards, P(s, a, s') is the probability of moving from state s to state s' after taking action a, and max_a is the maximum over all possible actions a.

To compute the V-values for an MDP, we start with an initial estimate of the V-values and update them iteratively using the Bellman equation until they converge to the true values. The update rule is:

$$V(s) \leftarrow R(s) + \gamma * \max_a (\sum_{s'} P(s, a, s') * V(s'))$$

We can use dynamic programming algorithms such as value iteration or policy iteration to compute the V-values.

We can use the Bellman equation to compute the V-values for each state in the maze. The V-values represent the expected cumulative reward that the robot can obtain if it starts from that state and follows an optimal policy thereafter. Complete the code below:

(**Note:** your final result can be slightly different from the result printed below and it's okay!)

In [4]:
def value_iteration(states, actions, rewards, transition_probs, discount, values, theta=1e-8):
    while True:
        delta = 0
        for state in states:
            old_value = values[state]
            new_values = []
            for action in actions:
                action_value = 0
                for next_state in states:
                    action_value += transition_probs[state][next_state][action] * (rewards[next_state] + discount * values[next_state])
                new_values.append(action_value)
            values[state] = max(new_values)
            delta = max(delta, abs(old_value - values[state]))
        if delta < theta:
            break
    return values

values = value_iteration(states, actions, rewards, transition_probs, discount, values)

print("V-values:")
for i in range(maze.shape[0]):
    for j in range(maze.shape[1]):
        print(values[i*maze.shape[1]+j], end=' ')
    print()

V-values:
10.901132229407896 12.24695103126669 15.47878533827593 19.563464808748375 15.478785342743663 
12.246951031266693 15.798417302076189 20.74114711905716 23.30178257078303 29.450864086830038 
15.478785338275934 18.436575216939698 23.30178257078303 29.450864086830038 34.30989706955881 
12.246951036372677 15.798417306656981 18.436575220488354 34.30989706955881 28.571428565746047 


## Section 4: Computing Q-values

In this section, we will explain how to compute Q-values for an MDP using the Bellman equation.

The Q-values represent the expected cumulative reward that the robot can obtain if it starts from a particular state and takes a particular action, and then follows an optimal policy thereafter. The Q-values can be computed using the Bellman equation as follows:

$$Q(s, a) = R(s, a) + \gamma * \sum_{s'} (P(s, a, s') * \max_{a'} (Q(s', a')))$$

where Q(s, a) is the Q-value of state-action pair (s, a), R(s, a) is the reward for taking action a in state s, γ is the discount factor that determines the importance of future rewards, P(s, a, s') is the probability of moving from state s to state s' after taking action a, max_a' is the maximum over all possible actions a' in state s', and sum_s' is the sum over all possible successor states s' of state s.

To compute the Q-values for an MDP, we start with an initial estimate of the Q-values and update them iteratively using the Bellman equation until they converge to the true values. The update rule is:

$$Q(s, a) \leftarrow R(s, a) + \gamma * \sum_{s'} (P(s, a, s') * \max_{a'} (Q(s', a')))$$

We can use dynamic programming algorithms such as Q-learning or SARSA to compute the Q-values.


We can use the Q-learning algorithm to compute the Q-values for each state-action pair in the maze. The Q-values represent the expected cumulative reward that the robot can obtain if it starts from a particular state and takes a particular action, and then follows an optimal policy thereafter. Complete the code below:

(**Note:** your final result can be slightly different from the result printed below and it's okay!)

In [5]:
def q_learning(states, actions, rewards, transition_probs, discount, q_values, theta=1e-8):
    while True:
        delta = 0
        for state in states:
            for action in actions:
                old_q_value = q_values[(state, action)]
                new_q_values = []
                for next_state in states:
                    next_q_values = []
                    for next_action in actions:
                        next_q_values.append(transition_probs[state][next_state][action] * (rewards[next_state] + discount * q_values[(next_state, next_action)]))
                    new_q_values.append(max(next_q_values))
                q_values[(state, action)] = sum(new_q_values)
                delta = max(delta, abs(old_q_value - q_values[(state, action)]))
        if delta < theta:
            break
    return q_values

q_values = q_learning(states, actions, rewards, transition_probs, discount, q_values)

print("Q-values:")
for state in states:
    for action in actions:
        print(f"Q({state}, {actions[action]}): {q_values[(state, action)]}")

Q-values:
Q(0, up): 8.951040797699095
Q(0, down): 10.901132235216561
Q(0, left): 2.0833274939023556
Q(0, right): 10.901132235765273
Q(1, up): 11.191997323455631
Q(1, down): 11.191997323455631
Q(1, left): 8.951040802483861
Q(1, right): 12.246951035723574
Q(2, up): 14.007662868931066
Q(2, down): 14.007662868931066
Q(2, left): 10.210895426094813
Q(2, right): 15.478785342175703
Q(3, up): 16.87187602339349
Q(3, down): 19.563464812160674
Q(3, left): 15.0025977107821
Q(3, right): 15.0025977107821
Q(4, up): 12.905437279460969
Q(4, down): 12.905437279460969
Q(4, left): 15.4787853455515
Q(4, right): 1.3930906810996353
Q(5, up): 8.951040802483861
Q(5, down): 12.246951035723573
Q(5, left): 11.191997327313686
Q(5, right): 11.191997327313686
Q(6, up): 11.341887896050164
Q(6, down): 15.798417306074645
Q(6, left): 11.579322108404279
Q(6, right): 14.136377823057051
Q(7, up): 15.108589118339344
Q(7, down): 20.741147122541882
Q(7, left): 18.423877040347104
Q(7, right): 20.26753456268599
Q(8, up): 16.1828

## Section 5: Visualizing the Optimal Policy

Now that we have computed the Q-values, we can use them to find the optimal policy, which is the sequence of actions that the robot should take in each state to maximize its expected reward. We can visualize the optimal policy as arrows in a grid, where each arrow corresponds to the action with the highest Q-value in the corresponding state. Complete the code below:

(**Note:** your final result can be slightly different from the result printed below and it's okay!)

In [7]:
def find_optimal_policy(states, actions, q_values):
    optimal_policy = {state: None for state in states}
    for state in states:
        optimal_action = max(actions, key=lambda action: q_values[(state, action)])
        optimal_policy[state] = actions[optimal_action]
    return optimal_policy

optimal_policy = find_optimal_policy(states, actions, q_values)

print("Optimal policy:")
for i in range(maze.shape[0]):
    for j in range(maze.shape[1]):
        print(optimal_policy[i*maze.shape[1]+j], end=' ')
    print()

print(q_values)

Optimal policy:
right right right down left 
down down down down down 
right right right right down 
up up up right down 
{(0, 0): 8.951040797699095, (0, 1): 10.901132235216561, (0, 2): 2.0833274939023556, (0, 3): 10.901132235765273, (1, 0): 11.191997323455631, (1, 1): 11.191997323455631, (1, 2): 8.951040802483861, (1, 3): 12.246951035723574, (2, 0): 14.007662868931066, (2, 1): 14.007662868931066, (2, 2): 10.210895426094813, (2, 3): 15.478785342175703, (3, 0): 16.87187602339349, (3, 1): 19.563464812160674, (3, 2): 15.0025977107821, (3, 3): 15.0025977107821, (4, 0): 12.905437279460969, (4, 1): 12.905437279460969, (4, 2): 15.4787853455515, (4, 3): 1.3930906810996353, (5, 0): 8.951040802483861, (5, 1): 12.246951035723573, (5, 2): 11.191997327313686, (5, 3): 11.191997327313686, (6, 0): 11.341887896050164, (6, 1): 15.798417306074645, (6, 2): 11.579322108404279, (6, 3): 14.136377823057051, (7, 0): 15.108589118339344, (7, 1): 20.741147122541882, (7, 2): 18.423877040347104, (7, 3): 20.26753456

In [8]:
actions_signs = {'up': '↑', 'down': '↓', 'left': '←', 'right': '→'}

policy_grid = [[' ' for _ in range(maze.shape[1])] for _ in range(maze.shape[0])]

for i in range(maze.shape[0]):
    for j in range(maze.shape[1]):
        state = i * maze.shape[1] + j
        policy_grid[i][j] = actions_signs[optimal_policy[state]]

for i in range(maze.shape[0]):
    for j in range(maze.shape[1]):
        if maze[i][j] == 2:
            policy_grid[i][j] = 'S'
        elif maze[i][j] == 3:
            policy_grid[i][j] = 'G'
        elif maze[i][j] == 1:
            policy_grid[i][j] = 'X'

for row in policy_grid:
    print(' '.join(row))

S → → ↓ ←
↓ X X ↓ X
→ → → → ↓
↑ X X X G
